In [1]:
import warnings
warnings.filterwarnings("ignore")


In [2]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR
df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,11.0,7.0,4.0,3.0,4.0,9.0,19.0,2022-01-29,4,17.0
1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,5.0,4.0,10.0,7.0,5.0,9.0,10.0,2022-01-30,4,9.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,3.0,...,8.0,7.0,8.0,5.0,5.0,10.0,0.0,2022-01-31,4,3.0
3,1.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,...,3.0,16.0,7.0,1.0,0.0,1.0,3.0,2022-02-01,4,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,2.0,...,3.0,8.0,3.0,0.0,4.0,4.0,3.0,2022-02-02,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-27,199,0.0
88290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-28,199,0.0
88291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-29,199,0.0
88292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-30,199,0.0


In [3]:
from datetime import datetime
from src.data_split import  train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32226, 674)
y_train.shape=(32226,)
X_test.shape=(56068, 674)
y_test.shape=(56068,)


In [4]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = KFold(n_splits=3)
    scores = []
        
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
         
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [5]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

[I 2024-09-11 12:41:24,215] A new study created in memory with name: no-name-69f4f1a2-e327-4bb7-ae9e-ea99ff7db22e
[I 2024-09-11 12:42:18,606] Trial 0 finished with value: 3.1908022606662008 and parameters: {'num_leaves': 135, 'feature_fraction': 0.49848705964467976, 'bagging_fraction': 0.7046934968403327, 'min_child_samples': 14}. Best is trial 0 with value: 3.1908022606662008.
[I 2024-09-11 12:42:58,859] Trial 1 finished with value: 3.699822928920836 and parameters: {'num_leaves': 75, 'feature_fraction': 0.7387137541420354, 'bagging_fraction': 0.922966237446124, 'min_child_samples': 76}. Best is trial 0 with value: 3.1908022606662008.
[I 2024-09-11 12:43:19,806] Trial 2 finished with value: 3.3961391915210797 and parameters: {'num_leaves': 21, 'feature_fraction': 0.36745350357540607, 'bagging_fraction': 0.8950781167745763, 'min_child_samples': 11}. Best is trial 0 with value: 3.1908022606662008.
[I 2024-09-11 12:44:02,972] Trial 3 finished with value: 3.7775606119225187 and parameters

In [6]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 135, 'feature_fraction': 0.49848705964467976, 'bagging_fraction': 0.7046934968403327, 'min_child_samples': 14}


In [7]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.49848705964467976, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.49848705964467976
[LightGBM] [Warning] bagging_fraction is set=0.7046934968403327, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7046934968403327
[LightGBM] [Warning] feature_fraction is set=0.49848705964467976, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.49848705964467976
[LightGBM] [Warning] bagging_fraction is set=0.7046934968403327, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7046934968403327
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.170820 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 154527
[LightGBM] [Info] Number of data points in the train set: 32226, number of used features: 675
[LightGBM] [Info] Start training from 

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x0000021D25037C10>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.7046934968403327,
                               feature_fraction=0.49848705964467976,
                               min_child_samples=14, num_leaves=135))])

In [8]:
from sklearn.metrics import  mean_absolute_error
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.49848705964467976, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.49848705964467976
[LightGBM] [Warning] bagging_fraction is set=0.7046934968403327, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7046934968403327
test_mae=2.7003


In [9]:
from src.plot import plot_one_sample

plot_one_sample(
    example_id=43,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

In [10]:
plot_one_sample(
    example_id=44,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)